# Duplicate Questions Classification

## Problem Statement- Duplicate Questions on Quora which were being answered by users and it wasting time of user.

In [ ]:
# Dataset link - https://www.kaggle.com/competitions/quora-question-pairs/overview

In [ ]:
# !pip install xgboost

In [ ]:
# Problems- crashing of google colab after multiple preprocessing and training of models
# Solution - saved preprocessed files to use again those which saved time to again preprocess data

In [ ]:
# Libraries import

import numpy as np
import pandas as pd
import os


import re
import string
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# importing ML Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb

#importing metrics score libraries
from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             classification_report,roc_auc_score)
from sklearn.metrics import precision_score,recall_score,f1_score

In [ ]:
# downloading necessary dependencies

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# reading csv file

data = pd.read_csv("train.csv")

In [ ]:
# read data

data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
data.shape

(404290, 6)

In [ ]:
data.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [ ]:
# removing null values rows
data.dropna(inplace=True)

In [ ]:
data.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [ ]:
# checking duplicate

data.duplicated().sum()

0

In [ ]:
# data info

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404287 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404287 non-null  int64 
 1   qid1          404287 non-null  int64 
 2   qid2          404287 non-null  int64 
 3   question1     404287 non-null  object
 4   question2     404287 non-null  object
 5   is_duplicate  404287 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


In [ ]:
# taking sample data for training as data is very large

sample_data= data.sample(frac=0.1)

In [ ]:
# sample data shape

sample_data.shape

(40429, 6)

In [ ]:
# index resest

sample_data.reset_index(inplace=True)

In [ ]:
# checking sample data

sample_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0


In [ ]:
# checking balnced or imbalanced dataset

sample_data['is_duplicate'].value_counts()

0    25506
1    14923
Name: is_duplicate, dtype: int64

In [ ]:
# initializing stemming

ps= PorterStemmer()

In [ ]:
# function for -removing secial characters, making lower case, removing stopwords and stemming

def text_preprocessing(x:str):

    x = str(x).lower().strip() #it will remove the white spaces
      # Replace certain special characters with their string equivalents
    x = x.replace('%', ' percent')
    x = x.replace('$', ' dollar ')
    x = x.replace('₹', ' rupee ')
    x = x.replace('€', ' euro ')
    x = x.replace('@', ' at ')

    # The pattern '[math]' appears around 900 times in the whole dataset.
    x = x.replace('[math]', '')

    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    x_decontracted = []

    for word in x.split():
        if word in contractions:
            word = contractions[word]

        x_decontracted.append(word)

    x = ' '.join(x_decontracted)
    x = x.replace("'ve", " have")
    x = x.replace("n't", " not")
    x = x.replace("'re", " are")
    x = x.replace("'ll", " will")
    x = re.sub('[^a-zA-Z0-9]', ' ', x)
    doc = x.split()
    doc = [ps.stem(word) for word in doc if not word in stopwords.words('english')]
    doc = ' '.join(doc)
    return doc


In [ ]:
# creating column of stemmed questions form question1
for i in tqdm(range(20)):
  sample_data['stem_question1']=sample_data['question1'].apply(text_preprocessing)

100%|██████████| 20/20 [23:58<00:00, 71.92s/it]


In [ ]:
# creating column of stemmed questions form question2
for i in tqdm(range(20)):
  sample_data['stem_question2']=sample_data['question2'].apply(text_preprocessing)

100%|██████████| 20/20 [24:12<00:00, 72.62s/it]


# Working with Stemmed Data

In [ ]:
# show data

sample_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,stem_question1,stem_question2
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex


In [ ]:
# saving stemmed data in to csv file

sample_data.to_csv('stemmed_data.csv')

In [ ]:
sample_data=pd.read_csv('stemmed_data.csv')

NameError: ignored

In [ ]:
sample_data.isnull().sum()

Unnamed: 0         0
index              0
id                 0
qid1               0
qid2               0
question1          0
question2          0
is_duplicate       0
stem_question1    10
stem_question2     5
dtype: int64

In [ ]:
# Dropping null values row

sample_data.dropna(inplace = True)

In [ ]:
# independent column

X= sample_data[['stem_question1','stem_question2']]

In [ ]:
# lable column

y =sample_data['is_duplicate']

In [ ]:
# Creating the Bag of Words model

cv = CountVectorizer(max_features=2500,binary = True,ngram_range=(1,2))
X1_vec,X2_vec = cv.fit_transform(X['stem_question1']).toarray(),cv.fit_transform(X['stem_question2']).toarray()

In [ ]:
# concatenate all arrays
X= np.concatenate((X1_vec,X2_vec),axis=1)

In [ ]:
# train test split

X_train_vec, X_test_vec, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Creating the Bag of Words model

# cv = CountVectorizer(max_features=5000,binary = True)
# x1_train_vec,x2_train_vec = cv.fit_transform(X_train['stem_question1']).toarray(),cv.fit_transform(X_train['stem_question2']).toarray()
# x1_test_vec,x2_test_vec =  cv.transform(X_test['stem_question1']).toarray(),cv.transform(X_test['stem_question2']).toarray()

In [ ]:
#shape of arrays
x1_train_vec.shape,x2_train_vec.shape

NameError: ignored

In [ ]:
# concat both arrays
# X_train_vec = np.concatenate((x1_train_vec,x2_train_vec),axis=1)

In [ ]:
# shape of concatenated arrray
X_train_vec.shape

(27079, 5000)

In [ ]:
# concat both test arrays
# X_test_vec = np.concatenate((x1_test_vec,x2_test_vec),axis=1)

In [ ]:
# Model Defining

# Logistic Regression classsification

LR = LogisticRegression(penalty='l2',C=0.001,max_iter=1000)

# Classification model using decision tree

DT= DecisionTreeClassifier()

# Classification model using Random Forest classifier

RF =RandomForestClassifier()

# Support vector classification Model

SV = SVC(probability=True)

# naive bayes
Naive_bayes = MultinomialNB()

#XGB Classifier
xg= xgb.XGBClassifier()

In [ ]:
# model accuracy, precision score,F1 score and classification report in Bag of words after stemming

Model_list ={"Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report={}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # fitting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Random Forest----- 

Accuracy Score on training data : 0.9929834927434543
Confusion Matrix on training data : 
 [[17067    48]
 [  142  9822]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.991749  0.997195  0.994465  17115.000000
1              0.995137  0.985749  0.990420   9964.000000
accuracy       0.992983  0.992983  0.992983      0.992983
macro avg      0.993443  0.991472  0.992443  27079.000000
weighted avg   0.992995  0.992983  0.992976  27079.000000



Accuracy Score on test data : 0.7177987704303493

Confusion Matrix on testing data : 
 [[6629 1751]
 [2013 2945]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.767068  0.791050  0.778874   8380.000000
1              0.627129  0.593990  0.610110   4958.000000
accuracy       0.717799  0.717799  0.717799      0.717799
macro avg      0.697099  0.692520  0.694492  13338.000

# Working without preprocessing on Questions

In [ ]:
# show data

sample_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,stem_question1,stem_question2,lem_question1,lem_question2
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find,people social medium get sent flashing message...,reacts socl2 give propan1ol find
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music,encryption email mobile number database good o...,need know basic music theory produce music
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format,different blu ray mkv format,difference m4v mp4 format
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...,anyone ask answer question quora,compatible quora policy ask question whose ans...
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex,enjoyable sex,enjoy sex


In [ ]:
# independent column

X= sample_data[['question1','question2']]

In [ ]:
# lable column

y =sample_data['is_duplicate']

In [ ]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Creating the Bag of Words model

cv = CountVectorizer(max_features=3000,binary = True)
x1_train_vec,x2_train_vec = cv.fit_transform(X_train['question1']).toarray(),cv.fit_transform(X_train['question2']).toarray()
x1_test_vec,x2_test_vec =  cv.transform(X_test['question1']).toarray(),cv.transform(X_test['question2']).toarray()

In [ ]:
# concat both arrays
X_train_vec = np.concatenate((x1_train_vec,x2_train_vec),axis=1)

In [ ]:
# concat both test arrays
X_test_vec = np.concatenate((x1_test_vec,x2_test_vec),axis=1)

In [ ]:
# shape of arrays

X_test_vec.shape,X_train_vec.shape

((13342, 6000), (27087, 6000))

In [ ]:
# model accuracy, precision score,F1 score and classification report in Bag of words without preprocessing

Model_list ={"Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report={}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # fitting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Logistic Regression----- 

Accuracy Score on training data : 0.6578063277587034
Confusion Matrix on training data : 
 [[16844   229]
 [ 9040   974]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.650749  0.986587  0.784226  17073.000000
1              0.809643  0.097264  0.173665  10014.000000
accuracy       0.657806  0.657806  0.657806      0.657806
macro avg      0.730196  0.541925  0.478946  27087.000000
weighted avg   0.709492  0.657806  0.558503  27087.000000



Accuracy Score on test data : 0.6382851146754609

Confusion Matrix on testing data : 
 [[8429    4]
 [4822   87]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.636103  0.999526  0.777440   8433.000000
1              0.956044  0.017723  0.034800   4909.000000
accuracy       0.638285  0.638285  0.638285      0.638285
macro avg      0.796073  0.508624  0.406120  133

### Accuracy achieved is 67.7% Till now after stemming

# Working with lemmatized data


In [ ]:
 # lemmetizer import and initialize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# function for -removing secial characters, making lower case, removing stopwords and lemmetizing

def text_preprocessing_lem(x:str):
        x = str(x).lower().strip() #it will remove the white spaces
          # Replace certain special characters with their string equivalents
        x = x.replace('%', ' percent')
        x = x.replace('$', ' dollar ')
        x = x.replace('₹', ' rupee ')
        x = x.replace('€', ' euro ')
        x = x.replace('@', ' at ')

        # The pattern '[math]' appears around 900 times in the whole dataset.
        x = x.replace('[math]', '')

        # Decontracting words
        # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
        # https://stackoverflow.com/a/19794953
        contractions = {
        "ain't": "am not",
        "aren't": "are not",
        "can't": "can not",
        "can't've": "can not have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "i would",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'll've": "i will have",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have"
        }

        x_decontracted = []

        for word in x.split():
            if word in contractions:
                word = contractions[word]

            x_decontracted.append(word)

        x = ' '.join(x_decontracted)
        x = x.replace("'ve", " have")
        x = x.replace("n't", " not")
        x = x.replace("'re", " are")
        x = x.replace("'ll", " will")
        x = re.sub('[^a-zA-Z0-9]', ' ', x)
        doc = x.split()
        doc = [lemmatizer.lemmatize(word) for word in doc if not word in stopwords.words('english')]
        doc = ' '.join(doc)
        return doc



In [ ]:
# creating column of lemmetized questions form question1
for i in tqdm(range(20)):
  sample_data['lem_question1']=sample_data['question1'].apply(text_preprocessing_lem)

100%|██████████| 20/20 [22:52<00:00, 68.63s/it]


In [ ]:
# creating column of lemmetized questions form question2
for i in tqdm(range(20)):
  sample_data['lem_question2']=sample_data['question2'].apply(text_preprocessing_lem)

100%|██████████| 20/20 [22:35<00:00, 67.76s/it]


In [ ]:
# show data
sample_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,stem_question1,stem_question2,lem_question1,lem_question2
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find,people social medium get sent flashing message...,reacts socl2 give propan1ol find
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music,encryption email mobile number database good o...,need know basic music theory produce music
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format,different blu ray mkv format,difference m4v mp4 format
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...,anyone ask answer question quora,compatible quora policy ask question whose ans...
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex,enjoyable sex,enjoy sex


In [ ]:
# saving lemmetized and stemmed data in to csv file

sample_data.to_csv('lemmatized_data.csv')

In [ ]:
# sample_data=pd.read_csv('lemmatized_data.csv')

In [ ]:
# independent column

X= sample_data[['lem_question1','lem_question2']]

In [ ]:
# lable column

y =sample_data['is_duplicate']

In [ ]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Creating the Bag of Words model

cv = CountVectorizer(max_features=2500,binary = True,ngram_range=(1,2))
x1_train_vec,x2_train_vec = cv.fit_transform(X_train['lem_question1']).toarray(),cv.fit_transform(X_train['lem_question2']).toarray()
x1_test_vec,x2_test_vec =  cv.transform(X_test['lem_question1']).toarray(),cv.transform(X_test['lem_question2']).toarray()

In [ ]:
# concat both arrays
X_train_vec = np.concatenate((x1_train_vec,x2_train_vec),axis=1)

In [ ]:
# concat both test arrays
X_test_vec = np.concatenate((x1_test_vec,x2_test_vec),axis=1)

In [ ]:
# model accuracy, precision score,F1-score and classification report in Bag of words on Lemmatized data

Model_list ={"Logistic Regression":LR,"Decision Tree": DT,
             "Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report= {}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # ftting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Logistic Regression----- 

Accuracy Score on training data : 0.6441466386089267
Confusion Matrix on training data : 
 [[17036    37]
 [ 9602   412]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.639538  0.997833  0.779483  17073.000000
1              0.917595  0.041142  0.078754  10014.000000
accuracy       0.644147  0.644147  0.644147      0.644147
macro avg      0.778566  0.519488  0.429119  27087.000000
weighted avg   0.742335  0.644147  0.520425  27087.000000



Accuracy Score on test data : 0.633788037775446

Confusion Matrix on testing data : 
 [[8433    0]
 [4886   23]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.633156  1.000000  0.775377   8433.000000
1              1.000000  0.004685  0.009327   4909.000000
accuracy       0.633788  0.633788  0.633788      0.633788
macro avg      0.816578  0.502343  0.392352  1334

## Feature Engineering

In [ ]:
# Feature Engineering

feature_data = sample_data.copy()

In [ ]:
# show data

feature_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,stem_question1,stem_question2,lem_question1,lem_question2
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find,people social medium get sent flashing message...,reacts socl2 give propan1ol find
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music,encryption email mobile number database good o...,need know basic music theory produce music
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format,different blu ray mkv format,difference m4v mp4 format
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...,anyone ask answer question quora,compatible quora policy ask question whose ans...
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex,enjoyable sex,enjoy sex


In [ ]:
# function to count words length

def count_words(x:str):
  x=x.split()
  x=len(x)
  return x

In [ ]:
# creating new columns asper length of words present in columns

feature_data['len_stem_question1']=feature_data['stem_question1'].apply(count_words)
feature_data['len_stem_question2']=feature_data['stem_question2'].apply(count_words)
feature_data['len_lem_question1']=feature_data['lem_question1'].apply(count_words)
feature_data['len_lem_question2']=feature_data['lem_question2'].apply(count_words)

In [ ]:
# show data
feature_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,stem_question1,stem_question2,lem_question1,lem_question2,len_stem_question1,len_stem_question2,len_lem_question1,len_lem_question2
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find,people social medium get sent flashing message...,reacts socl2 give propan1ol find,11,5,11,5
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music,encryption email mobile number database good o...,need know basic music theory produce music,7,7,7,7
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format,different blu ray mkv format,difference m4v mp4 format,5,4,5,4
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...,anyone ask answer question quora,compatible quora policy ask question whose ans...,5,12,5,12
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex,enjoyable sex,enjoy sex,2,2,2,2


In [ ]:
# function to find common words from question1 and question2

def common_words_stem(row):
    w1 = set(map(lambda word: word.lower().strip(), row['stem_question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['stem_question2'].split(" ")))
    return len(w1 & w2)

def common_words_lem(row):
    w1 = set(map(lambda word: word.lower().strip(), row['lem_question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['lem_question2'].split(" ")))
    return len(w1 & w2)

In [ ]:
# common word column
feature_data['stem_word_common'] = feature_data.apply(common_words_stem, axis=1)
feature_data['lem_word_common'] = feature_data.apply(common_words_lem, axis=1)

In [ ]:
# show data

feature_data.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,stem_question1,stem_question2,lem_question1,lem_question2,len_stem_question1,len_stem_question2,len_lem_question1,len_lem_question2,stem_word_common,lem_word_common
0,17993,17993,34118,34119,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find,people social medium get sent flashing message...,reacts socl2 give propan1ol find,11,5,11,5,0,0
1,227279,227279,336072,336073,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music,encryption email mobile number database good o...,need know basic music theory produce music,7,7,7,7,0,0
2,282293,282293,402197,402198,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format,different blu ray mkv format,difference m4v mp4 format,5,4,5,4,2,1
3,255244,255244,366782,226672,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...,anyone ask answer question quora,compatible quora policy ask question whose ans...,5,12,5,12,4,4
4,89406,89406,150214,150215,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex,enjoyable sex,enjoy sex,2,2,2,2,2,1


In [ ]:
# dropping unnecessary columns

feature_data.drop(columns=['index','qid1','qid2'],inplace=True)

In [ ]:
feature_data

,id,question1,question2,is_duplicate,stem_question1,stem_question2,lem_question1,lem_question2,len_stem_question1,len_stem_question2,len_lem_question1,len_lem_question2,stem_word_common,lem_word_common
0,17993,What do people do with social media when they ...,A reacts with socl2 and give propan1ol?find wh...,0,peopl social media get sent flash messag caus ...,react socl2 give propan1ol find,people social medium get sent flashing message...,reacts socl2 give propan1ol find,11,5,11,5,0,0
1,227279,Is encryption of email and mobile number in da...,Do I need to know basic music theory to produc...,0,encrypt email mobil number databas good option,need know basic music theori produc music,encryption email mobile number database good o...,need know basic music theory produce music,7,7,7,7,0,0
2,282293,What is the different between blu-ray and mkv ...,What are the differences between an .m4v and a...,0,differ blu ray mkv format,differ m4v mp4 format,different blu ray mkv format,difference m4v mp4 format,5,4,5,4,2,1
3,255244,Can anyone ask and answer a same question in Q...,Is it compatible with Quora policies to ask a ...,1,anyon ask answer question quora,compat quora polici ask question whose answer ...,anyone ask answer question quora,compatible quora policy ask question whose ans...,5,12,5,12,4,4
4,89406,What's so enjoyable about sex?,What do you enjoy about sex?,0,enjoy sex,enjoy sex,enjoyable sex,enjoy sex,2,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40424,84849,What is the different between a tutor and a st...,What is the difference between a tutor and a t...,0,differ tutor student teacher,differ tutor teacher,different tutor student teacher,difference tutor teacher,4,3,4,3,3,2
40425,339900,Is it possible to hack into a wifi modem to ge...,Is it possible to hack a Comcast cable modem?,0,possibl hack wifi modem get free internet,possibl hack comcast cabl modem,possible hack wifi modem get free internet,possible hack comcast cable modem,7,5,7,5,3,3
40426,351933,How can I hack wechat using phone?,How can I hack WeChat messages without using t...,0,hack wechat use phone,hack wechat messag without use phone,hack wechat using phone,hack wechat message without using phone,4,6,4,6,4,4
40427,61947,What are the effects of demonetization in India?,How is demonetization helping India?,1,effect demonet india,demonet help india,effect demonetization india,demonetization helping india,3,3,3,3,2,2


In [ ]:
# saving extracted features data
feature_data.to_csv('featured_data.csv')

In [ ]:
feature_data= pd.read_csv('featured_data.csv')

In [ ]:
feature_data.dropna(inplace=True)

# Woking with Stemmed and Featured data

In [ ]:
# independent column

X= feature_data[['stem_question1','stem_question2']]

In [ ]:
# lable column

y =feature_data['is_duplicate']

In [ ]:
# Creating the Bag of Words model

cv = CountVectorizer(max_features=2500,binary = True,ngram_range=(1,2))
X1_vec,X2_vec = cv.fit_transform(X['stem_question1']).toarray(),cv.fit_transform(X['stem_question2']).toarray()

In [ ]:
# Converting column values in to array

feature_arr= feature_data[['len_stem_question1','len_stem_question2','stem_word_common']].to_numpy()

In [ ]:
feature_arr.shape

(40417, 3)

In [ ]:
# concatenate all arrays
X= np.concatenate((X1_vec,X2_vec,feature_arr),axis=1)


In [ ]:
X.shape

(40417, 5003)

In [ ]:
# train test split

X_train_vec, X_test_vec, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# model accuracy, precision score,F1-score and classification report in Bag of words on Lemmatized data

Model_list ={"Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report= {}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # ftting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Random Forest----- 

Accuracy Score on training data : 0.9971933970973818
Confusion Matrix on training data : 
 [[17058    57]
 [   19  9945]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.998887  0.996670  0.997777  17115.000000
1              0.994301  0.998093  0.996194   9964.000000
accuracy       0.997193  0.997193  0.997193      0.997193
macro avg      0.996594  0.997381  0.996985  27079.000000
weighted avg   0.997200  0.997193  0.997195  27079.000000



Accuracy Score on test data : 0.7628579997001049

Confusion Matrix on testing data : 
 [[7229 1151]
 [2012 2946]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.782275  0.862649  0.820498   8380.000000
1              0.719063  0.594191  0.650690   4958.000000
accuracy       0.762858  0.762858  0.762858      0.762858
macro avg      0.750669  0.728420  0.735594  13338.000

In [ ]:
# Creating the TFIDF model
tv = TfidfVectorizer(max_features = 5000, ngram_range=(1,2))
X1_vec,X2_vec = tv.fit_transform(X['stem_question1']).toarray(),tv.fit_transform(X['stem_question2']).toarray()

In [ ]:
# Converting column values in to array

feature_arr= feature_data[['len_stem_question1','len_stem_question2','stem_word_common']].to_numpy()

In [ ]:
feature_arr.shape

(40417, 3)

In [ ]:
# concatenate all arrays
X= np.concatenate((X1_vec,X2_vec,feature_arr),axis=1)

In [ ]:
X.shape

(40417, 10003)

In [ ]:
# train test split

X_train_vec, X_test_vec, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# TFIDF  - model accuracy, precision score,F1-score and classification report in Bag of words on Lemmatized data

Model_list ={"Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report= {}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # ftting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Random Forest----- 

Accuracy Score on training data : 0.998412053620887
Confusion Matrix on training data : 
 [[17088    27]
 [   16  9948]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.999065  0.998422  0.998743  17115.000000
1              0.997293  0.998394  0.997843   9964.000000
accuracy       0.998412  0.998412  0.998412      0.998412
macro avg      0.998179  0.998408  0.998293  27079.000000
weighted avg   0.998413  0.998412  0.998412  27079.000000



Accuracy Score on test data : 0.763907632328685

Confusion Matrix on testing data : 
 [[7425  955]
 [2194 2764]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.771910  0.886038  0.825046   8380.000000
1              0.743211  0.557483  0.637087   4958.000000
accuracy       0.763908  0.763908  0.763908      0.763908
macro avg      0.757560  0.721761  0.731066  13338.00000

# Working with lemmetized and featured data

In [ ]:
# independent column

X= feature_data[['lem_question1','lem_question2']]

In [ ]:
# lable column

y =feature_data['is_duplicate']

In [ ]:
# Creating the Bag of Words model

cv = CountVectorizer(max_features=2500,binary = True,ngram_range=(1,2))
X1_vec,X2_vec = cv.fit_transform(X['lem_question1']).toarray(),cv.fit_transform(X['lem_question2']).toarray()

In [ ]:
# Converting column values in to array

feature_arr= feature_data[['len_lem_question1','len_lem_question2','lem_word_common']].to_numpy()

In [ ]:
feature_arr.shape

(40417, 3)

In [ ]:
# concatenate all arrays
X= np.concatenate((X1_vec,X2_vec,feature_arr),axis=1)

In [ ]:
# train test split

X_train_vec, X_test_vec, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# model accuracy, precision score,F1-score and classification report in Bag of words on Lemmatized data

Model_list ={"Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report= {}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # ftting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Random Forest----- 

Accuracy Score on training data : 0.9969718231840171
Confusion Matrix on training data : 
 [[17071    44]
 [   38  9926]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.997779  0.997429  0.997604  17115.000000
1              0.995587  0.996186  0.995886   9964.000000
accuracy       0.996972  0.996972  0.996972      0.996972
macro avg      0.996683  0.996808  0.996745  27079.000000
weighted avg   0.996972  0.996972  0.996972  27079.000000



Accuracy Score on test data : 0.7681811366021892

Confusion Matrix on testing data : 
 [[7298 1082]
 [2010 2948]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.784057  0.870883  0.825192   8380.000000
1              0.731514  0.594595  0.655986   4958.000000
accuracy       0.768181  0.768181  0.768181      0.768181
macro avg      0.757785  0.732739  0.740589  13338.000

In [ ]:
# Creating the TFIDF model
tv = TfidfVectorizer(max_features = 5000, ngram_range=(1,2))
X1_vec,X2_vec = tv.fit_transform(X['lem_question1']).toarray(),tv.fit_transform(X['lem_question2']).toarray()

In [ ]:
# Converting column values in to array

feature_arr= feature_data[['len_lem_question1','len_lem_question2','lem_word_common']].to_numpy()

In [ ]:
# concatenate all arrays
X= np.concatenate((X1_vec,X2_vec,feature_arr),axis=1)

In [ ]:
# train test split

X_train_vec, X_test_vec, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# model accuracy, precision score,F1-score and classification report in Bag of words on Lemmatized data

Model_list ={"Random Forest":RF,"Naive bayes":Naive_bayes}

accuracy_report= {}
auc_report = {}
precision_report ={}
f1_report= {}

for model_name,model in Model_list.items():
    model.fit(X_train_vec, y_train)                              # ftting pipeline to train data
    y_pred_on_train=model.predict(X_train_vec)                     # predicting y_pred from training data
    y_pred_on_test=model.predict(X_test_vec)                          # predicting y_pred from test data
    y_pred_prob_test =model.predict_proba(X_test_vec)[:,1]            # predicting y_pred probability from test data
    # n_errors_train =  (y_pred_on_train!=y_train).sum()        # no. of errors in training data
    # n_errors_test =  (y_pred_on_test!=y_test).sum()              # no. of errors in  testing data
    training_report = classification_report(y_train, y_pred_on_train,output_dict=True)  # classification report from training data
    df_classification_report_train = pd.DataFrame(training_report).transpose()             # converting report to dataframe
    testing_report = classification_report(y_test, y_pred_on_test,output_dict=True)        # clasification report from testing data
    df_classification_report_test = pd.DataFrame(testing_report).transpose()               # converting report to dataframe
    auc_score_test=roc_auc_score(y_test, y_pred_prob_test)                                 # auc score on testing data
    auc_report[model_name]= auc_score_test

    accuracy_sc=accuracy_score(y_test, y_pred_on_test)
    accuracy_report[model_name]=accuracy_sc

    precision = precision_score(y_test, y_pred_on_test)
    precision_report[model_name] =precision

    F1 = f1_score(y_test, y_pred_on_test)
    f1_report[model_name] =F1


    print(f"-----Metrics score of {model_name}----- \n")        #printing model name
    print(f"Accuracy Score on training data : {accuracy_score(y_train, y_pred_on_train)}") # printing accuracy score on train data
    print(f"Confusion Matrix on training data : \n {confusion_matrix(y_train, y_pred_on_train)}") #training data confusion matrix
    print(f"Classification Report on training data :\n {df_classification_report_train}\n")         # classification report on train data


    print("\n")     # next line
    print(f"Accuracy Score on test data : {accuracy_score(y_test, y_pred_on_test)}\n")  # printing accuracy score on testing data
    print(f"Confusion Matrix on testing data : \n {confusion_matrix(y_test, y_pred_on_test)}") #training data confusion matrix
    print(f"Classification Report on testing data : \n {df_classification_report_test}\n")   # classification report printing
    print(f"Auc_score on testing data : \n {auc_score_test}")  #auc score printing
    print(f"Precision Score on test data : {precision_score(y_test, y_pred_on_test)}\n") # precision score on test data
    print(f"F1 Score on test data : {f1_score(y_test, y_pred_on_test)}\n") # precision score on test data



best_model_auc = max(auc_report.values())
best_model = [i for i,j in auc_report.items() if j==best_model_auc]
print(f"best model is :  {best_model} with auc score {best_model_auc}")

best_model_accuracy = max(accuracy_report.values())
best_model_accuracy_name = [i for i,j in accuracy_report.items() if j==best_model_accuracy]
print(f"best model_accuracy is :  {best_model_accuracy_name} with accuracy score {best_model_accuracy}")


best_model_precision = max(precision_report.values())
best_model_prec_name = [i for i,j in precision_report.items() if j==best_model_precision]
print(f"best model_precision is :  {best_model_prec_name} with precision score {best_model_precision}")


best_model_f1_score = max(f1_report.values())
best_model_f1_score_name = [i for i,j in f1_report.items() if j==best_model_f1_score ]
print(f"best model_f1 is :  {best_model_f1_score_name} with f1 score {best_model_f1_score}")






-----Metrics score of Random Forest----- 

Accuracy Score on training data : 0.9983381956497654
Confusion Matrix on training data : 
 [[17088    27]
 [   18  9946]]
Classification Report on training data :
               precision    recall  f1-score       support
0              0.998948  0.998422  0.998685  17115.000000
1              0.997293  0.998193  0.997743   9964.000000
accuracy       0.998338  0.998338  0.998338      0.998338
macro avg      0.998120  0.998308  0.998214  27079.000000
weighted avg   0.998339  0.998338  0.998338  27079.000000



Accuracy Score on test data : 0.7666066876593193

Confusion Matrix on testing data : 
 [[7388  992]
 [2121 2837]]
Classification Report on testing data : 
               precision    recall  f1-score       support
0              0.776948  0.881623  0.825982   8380.000000
1              0.740925  0.572207  0.645727   4958.000000
accuracy       0.766607  0.766607  0.766607      0.766607
macro avg      0.758936  0.726915  0.735855  13338.000

### Feature engineering played important role, also here splitting of data was after bag of words it has increased accuracy up to 76.8% here, let's see what else feature we can extract

As Dataset is small as compared to whole data, accuracy will be increeasing as we work with whole data.